# 01 ORM
오늘부터는 ORM 사용한다.
얘네는 또 독립적으로 움직인다.

## 01.01 ORM 생성

In [1]:
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import relationship
from sqlalchemy import and_
from sqlalchemy.orm import declarative_base

In [2]:
base = declarative_base()

In [3]:
engine = create_engine('sqlite:///:memory:', echo=True)

base.metadata.tables
위에 코어에서 관리하는 Table 객체들이다.

base.registry
위에 오브젝트들이 등록이 된다.

In [4]:
base.metadata.tables

FacadeDict({})

In [5]:
base.metadata.remove(base.metadata.tables['USER'])
base.registry.dispose()

KeyError: 'USER'

In [6]:
class User(base):
    __tablename__ = 'USER'
    #object에서 접근할 수 있는 테이블 이름이며, DBMS에서 사용될 TABLE 이름이다.
    pk = Column('PK', Integer, primary_key=True)
    #오브젝트에서 사용할 수 있는 pk칼럼 객체 => 실제 DBMS에서 해당 TABLe에 사용되는 Column 정보가 된다.
    name = Column('NAME', Text)
    
    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}'

In [7]:
user1 = User(name='Philip')

In [8]:
type(user1), type(base.metadata.tables['USER'])
#왼쪽은 우리가 정의한 클래스 객체, 오른쪽은 테이블 객체

(__main__.User, sqlalchemy.sql.schema.Table)

In [11]:
base.metadata.create_all(engine)

2023-09-07 09:45:43,820 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:45:43,823 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-07 09:45:43,824 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:45:43,825 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2023-09-07 09:45:43,827 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:45:43,829 INFO sqlalchemy.engine.Engine 
CREATE TABLE "USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2023-09-07 09:45:43,830 INFO sqlalchemy.engine.Engine [no key 0.00102s] ()
2023-09-07 09:45:43,831 INFO sqlalchemy.engine.Engine COMMIT


여기까지는 어제 한 거를 클래스로 한번 감싼것과 큰 차이가 없다. 이제는 세션이라는 것을 사용할것이다. 쿼리를 날리든 어쩌든 통신을 하는 것을 세션이 맡아줄것이다. 어제는 con(connection) 객체의 execute를 통해 했었다. 이제 세션은 객체가 DBMS와 맵핑되어 있음을 알고 있고 객체단에서 조작이 될때 뒷단에서 일어나야 하는 일을 알아서 해준다. 따라서 engine.connect().execute()는 이제 더이상 몰라도 된다. 세션이 이 일을 처리하기 때문이다.

## 01.02 Session

In [13]:
Session = sessionmaker(engine)
sess = Session()
#in memory에 있는 오브젝트와 RDBMS간의 창구 역할을 한다.

In [14]:
sess.add(user1)
#세션아 얘를 관리해라

In [15]:
sess.commit()
#실질적으로 이 커밋의 내용은 세션이라고 하는애가 관찰하고 있는 오브젝트들을 반영하라 라는 의미이다.
#레이지커넥팅에 의해 이제 인서트하고 커밋한다.

2023-09-07 09:45:46,206 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:45:46,209 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?)
2023-09-07 09:45:46,210 INFO sqlalchemy.engine.Engine [generated in 0.00115s] ('Philip',)
2023-09-07 09:45:46,213 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
user1
#user1이 등록되어 있는 객체임을 알고 있어서, SELECT를 하는 것임.

2023-09-07 09:45:46,558 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:45:46,563 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:45:46,564 INFO sqlalchemy.engine.Engine [generated in 0.00107s] (1,)


PK:1, NAME:Philip

In [17]:
sess.is_modified(user1)

False

In [18]:
sess.dirty

IdentitySet([])

In [19]:
user1.name = "PhilipSung"

In [20]:
user1

PK:1, NAME:PhilipSung

In [21]:
sess.dirty

IdentitySet([PK:1, NAME:PhilipSung])

In [22]:
sess.is_modified(user1)

True

In [23]:
sess.commit()

2023-09-07 09:45:48,690 INFO sqlalchemy.engine.Engine UPDATE "USER" SET "NAME"=? WHERE "USER"."PK" = ?
2023-09-07 09:45:48,691 INFO sqlalchemy.engine.Engine [generated in 0.00140s] ('PhilipSung', 1)
2023-09-07 09:45:48,693 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
user1
#값이 변경된 것이 반영되었으므로, SELECT를 실행한다.

2023-09-07 09:45:49,046 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:45:49,048 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:45:49,049 INFO sqlalchemy.engine.Engine [cached since 2.485s ago] (1,)


PK:1, NAME:PhilipSung

In [25]:
sess.add_all([User(name='2'), User(name='3')])

In [26]:
sess.commit()

2023-09-07 09:45:49,746 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 09:45:49,748 INFO sqlalchemy.engine.Engine [generated in 0.00018s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('2',)
2023-09-07 09:45:49,750 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 09:45:49,751 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('3',)
2023-09-07 09:45:49,752 INFO sqlalchemy.engine.Engine COMMIT


In [27]:
userList = sess.query(User).all()

2023-09-07 09:45:49,965 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:45:49,967 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER"
2023-09-07 09:45:49,968 INFO sqlalchemy.engine.Engine [generated in 0.00131s] ()


In [28]:
len(userList)

3

In [29]:
user1 is userList[0]

True

In [30]:
userList[1].name = 'Philip2'

In [31]:
sess.dirty

IdentitySet([PK:2, NAME:Philip2])

In [32]:
#값이 변경되었음을 알고 있다.

In [33]:
sess.commit()

2023-09-07 09:45:51,837 INFO sqlalchemy.engine.Engine UPDATE "USER" SET "NAME"=? WHERE "USER"."PK" = ?
2023-09-07 09:45:51,839 INFO sqlalchemy.engine.Engine [cached since 3.149s ago] ('Philip2', 2)
2023-09-07 09:45:51,841 INFO sqlalchemy.engine.Engine COMMIT


In [34]:
userList[1]

2023-09-07 09:45:52,248 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:45:52,251 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:45:52,252 INFO sqlalchemy.engine.Engine [cached since 5.688s ago] (2,)


PK:2, NAME:Philip2

즉 세션의dirty에 값이 들어있으면, 데이터베이스와 객체간에 Sync가 맞지 않다는 것이다.
이제부터는 객체만 관리하면 되는것이다.

In [35]:
[user for user in userList if user.name == '3'][0] is userList[2]

2023-09-07 09:45:52,922 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:45:52,923 INFO sqlalchemy.engine.Engine [cached since 6.36s ago] (1,)
2023-09-07 09:45:52,925 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-07 09:45:52,926 INFO sqlalchemy.engine.Engine [cached since 6.362s ago] (3,)


True

In [36]:
#위에서는 객체만으로도 연산했지만, 세션한테 가져오라고 할수도 있다.
sess.query(User).where(User.name == '3').one() is userList[2]

2023-09-07 09:45:53,250 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."NAME" = ?
2023-09-07 09:45:53,253 INFO sqlalchemy.engine.Engine [generated in 0.00187s] ('3',)


True

add 했을 때 pending, 작업할 목록에 들고만 있다가 commit해줘야 실제 작업을 한다.

dirty : 값의 싱크가 틀어졌을때

is_modified : 값이 변했을 때

## ?

In [31]:
engine.dispose()
base.metadata.clear()
base.registry.dispose()

In [32]:
base.metadata.tables

FacadeDict({})

In [33]:
#클래스만 날린다.
base.registry.dispose()

CITY, SUPPLIER, PART, SELLS => ORM으로 만들어보세요<br>
네개의 클래스가 필요할 것

In [34]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [35]:
Session = sessionmaker(engine)
sess = Session()

In [36]:
class City(base):
    __tablename__ = 'CITY'
    __table_args__ = {'extend_existing':True}
    pk = Column('CNO', Integer, primary_key=True)
    name = Column('CNAME', Text)
    nodes = relationship('Supplier', back_populates='city', uselist=True)
    
    def __repr__(self):
        return f'CNO: {self.pk}, CNAME: {self.name}'
    

In [37]:
class Supplier(base):
    __tablename__ = 'SUPPLIER'
    __table_args__ = {'extend_existing':True}
    pk = Column('SNO', Integer, primary_key=True)
    name = Column('SNAME', Text)
    fk = Column('CNO', Integer, ForeignKey('CITY.CNO'))
    city = relationship('City', back_populates='nodes')
    
    def __repr__(self):
        return f'SNO: {self.pk}, SNAME: {self.name}, CNO: {self.fk}'

In [38]:
class Part(base):
    __tablename__ = 'PART'
    __table_args__ = {'extend_existing':True}
    pk = Column('PNO', Integer, primary_key=True)
    name = Column('PNAME', Text)
    
    def __repr__(self):
        return f'PNO: {self.pk}, PNAME: {self.name}'

In [39]:
class Sells(base):
    __tablename__ = 'SELLS'
    __table_args__ = {'extend_existing':True}
    pk = Column('SELLNO', Integer, primary_key=True)
    fk1 = Column('SNO',Integer, ForeignKey('SUPPLIER.SNO'))
    fk2 = Column('PNO', Integer, ForeignKey('PART.PNO'))
    price = Column('PRICE', Integer)
    
    def __repr(self):
        return f'SNO: {self.fk1}, PNO: {self.fk2}, PRICE: {self.price}'

In [40]:
base.metadata.create_all(engine)

2023-09-07 10:37:12,443 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:37:12,445 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CITY")
2023-09-07 10:37:12,448 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:37:12,450 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CITY")
2023-09-07 10:37:12,452 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:37:12,454 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SUPPLIER")
2023-09-07 10:37:12,456 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:37:12,458 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SUPPLIER")
2023-09-07 10:37:12,459 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:37:12,461 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PART")
2023-09-07 10:37:12,462 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:37:12,464 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PART")
2023-09-07 10:37:12,465 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-0

In [41]:
sess.add_all([
    City(name="City1"),
    City(name="City2"),
    Part(name="Part1"), 
    Part(name="Part2"), 
    Supplier(name="Supplier1"), 
    Supplier(name="Supplier2"), 
              ])

In [42]:
sess.commit()

2023-09-07 10:37:13,725 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:37:13,730 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("CNAME") VALUES (?) RETURNING "CNO"
2023-09-07 10:37:13,732 INFO sqlalchemy.engine.Engine [generated in 0.00033s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('City1',)
2023-09-07 10:37:13,734 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("CNAME") VALUES (?) RETURNING "CNO"
2023-09-07 10:37:13,734 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('City2',)
2023-09-07 10:37:13,738 INFO sqlalchemy.engine.Engine INSERT INTO "PART" ("PNAME") VALUES (?) RETURNING "PNO"
2023-09-07 10:37:13,739 INFO sqlalchemy.engine.Engine [generated in 0.00019s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Part1',)
2023-09-07 10:37:13,742 INFO sqlalchemy.engine.Engine INSERT INTO "PART" ("PNAME") VALUES (?) RETURNING "PNO"
2023-09-07 10:37:13,745 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered

In [43]:
sess.query(Supplier).filter(Supplier.pk == 1).one()

2023-09-07 10:37:13,937 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:37:13,941 INFO sqlalchemy.engine.Engine SELECT "SUPPLIER"."SNO" AS "SUPPLIER_SNO", "SUPPLIER"."SNAME" AS "SUPPLIER_SNAME", "SUPPLIER"."CNO" AS "SUPPLIER_CNO" 
FROM "SUPPLIER" 
WHERE "SUPPLIER"."SNO" = ?
2023-09-07 10:37:13,942 INFO sqlalchemy.engine.Engine [generated in 0.00162s] (1,)


SNO: 1, SNAME: Supplier1, CNO: None

**테이블구현예제 끝**
***

## ORM Relationship

orm에 있는 오브젝트들 독립적으로 돌아가는데, DBMS에 테이블 여러개를 관계를 맺도록 pk와 fk를 이용해서 만들어놨다. 이거와 마찬가지로 ORM Core에도 Table.ForeignKey를 이용해 만들었다. 이제 orm단에서도 뒷단과 별개로 관계를 만들어줄 수 있다. 지금처럼 아무리 객체를 만들었어오. Oobject.pk나 Objejct.fk 를 필터하거나 연산을 해야만 가져올 수 있는데,

Object1.childnodes[] ==> Object2가 되도록 구조를 만들어 줄 수 있다.

In [48]:
city = sess.query(City).where(City.pk==1).one()

2023-09-07 10:39:07,585 INFO sqlalchemy.engine.Engine SELECT "CITY"."CNO" AS "CITY_CNO", "CITY"."CNAME" AS "CITY_CNAME" 
FROM "CITY" 
WHERE "CITY"."CNO" = ?
2023-09-07 10:39:07,587 INFO sqlalchemy.engine.Engine [cached since 58.43s ago] (1,)


In [49]:
city.nodes[0] is sess.query(Supplier).where(Supplier.fk==1).one()

IndexError: list index out of range

아무튼 seel.supplier.city.nodes.sells 이런식으로 연결이 주루룩 된다.

### 초기화해서 Relationship 다시

In [162]:
engine.dispose()
sess.close()
base.registry.dispose()
base.metadata.clear()

In [163]:
engine = create_engine('sqlite:///:memory:', echo=True)
base = declarative_base()
Session = sessionmaker(engine)
sess = Session()

In [164]:
class Artist(base):
    __tablename__ = 'ARTIST'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    albums = relationship('Album', back_populates='artist', uselist=True)
    
    def addAlbum(self, name):
        album = Album(name = name)
        sess.add(album)
        sess.commit()
        self.albums.append(album)
    
    def __repr__(self):
        return f'{self.name}, {len(self.albums)}'
    
class Album(base):
    __tablename__ = 'ALBUM'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', Integer, ForeignKey('ARTIST.PK'))
    artist = relationship('Artist', back_populates='albums')

#내가 쓴 삭제 코드
#     def delAlbum(pk):
#         album = sess.query(Album).where(Album.pk == 1).one()
#         sess.delete(album)
#         sess.commit()
    
    def __repr__(self):
        return f'PK :{self.pk} - {self.name}, {self.artist.name}'
    

In [165]:
base.metadata.create_all(engine)

2023-09-07 11:20:45,657 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:20:45,659 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ARTIST")
2023-09-07 11:20:45,661 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:20:45,664 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ARTIST")
2023-09-07 11:20:45,666 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:20:45,668 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ALBUM")
2023-09-07 11:20:45,670 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:20:45,672 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ALBUM")
2023-09-07 11:20:45,673 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:20:45,677 INFO sqlalchemy.engine.Engine 
CREATE TABLE "ARTIST" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2023-09-07 11:20:45,680 INFO sqlalchemy.engine.Engine [no key 0.00294s] ()
2023-09-07 11:20:45,685 INFO sqlalchemy.engine.Engine 
CREATE TABLE "ALBUM" (
	"PK" INTEGER NOT N

In [166]:
artist1 = Artist(name='가수1')
sess.add(artist1)
sess.commit()

2023-09-07 11:20:46,419 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:20:46,425 INFO sqlalchemy.engine.Engine INSERT INTO "ARTIST" ("NAME") VALUES (?)
2023-09-07 11:20:46,426 INFO sqlalchemy.engine.Engine [generated in 0.00147s] ('가수1',)
2023-09-07 11:20:46,430 INFO sqlalchemy.engine.Engine COMMIT


In [167]:
artist1

2023-09-07 11:20:46,890 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:20:46,894 INFO sqlalchemy.engine.Engine SELECT "ARTIST"."PK" AS "ARTIST_PK", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."PK" = ?
2023-09-07 11:20:46,895 INFO sqlalchemy.engine.Engine [generated in 0.00170s] (1,)
2023-09-07 11:20:46,903 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE ? = "ALBUM"."FK"
2023-09-07 11:20:46,904 INFO sqlalchemy.engine.Engine [generated in 0.00190s] (1,)


가수1, 0

In [168]:
artist1.addAlbum('앨범1')

2023-09-07 11:20:48,970 INFO sqlalchemy.engine.Engine INSERT INTO "ALBUM" ("NAME", "FK") VALUES (?, ?)
2023-09-07 11:20:48,971 INFO sqlalchemy.engine.Engine [generated in 0.00143s] ('앨범1', None)
2023-09-07 11:20:48,976 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:20:48,978 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:20:48,980 INFO sqlalchemy.engine.Engine SELECT "ARTIST"."PK" AS "ARTIST_PK", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."PK" = ?
2023-09-07 11:20:48,981 INFO sqlalchemy.engine.Engine [cached since 2.087s ago] (1,)
2023-09-07 11:20:48,986 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE ? = "ALBUM"."FK"
2023-09-07 11:20:48,988 INFO sqlalchemy.engine.Engine [cached since 2.085s ago] (1,)


In [169]:
artist1.albums[0]

2023-09-07 11:20:51,249 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME" 
FROM "ALBUM" 
WHERE "ALBUM"."PK" = ?
2023-09-07 11:20:51,252 INFO sqlalchemy.engine.Engine [generated in 0.00175s] (1,)
2023-09-07 11:20:51,256 INFO sqlalchemy.engine.Engine UPDATE "ALBUM" SET "FK"=? WHERE "ALBUM"."PK" = ?
2023-09-07 11:20:51,256 INFO sqlalchemy.engine.Engine [generated in 0.00150s] (1, 1)
2023-09-07 11:20:51,263 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."PK" = ?
2023-09-07 11:20:51,265 INFO sqlalchemy.engine.Engine [generated in 0.00187s] (1,)


PK :1 - 앨범1, 가수1

In [170]:
artist1

가수1, 1

In [171]:
album = Album(name = "MyAlbum", fk = 1)
sess.add(album)
sess.commit()

2023-09-07 11:20:59,035 INFO sqlalchemy.engine.Engine INSERT INTO "ALBUM" ("NAME", "FK") VALUES (?, ?)
2023-09-07 11:20:59,037 INFO sqlalchemy.engine.Engine [cached since 10.07s ago] ('MyAlbum', 1)
2023-09-07 11:20:59,039 INFO sqlalchemy.engine.Engine COMMIT


In [175]:
sess.query(Album).all()

2023-09-07 11:21:31,383 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:21:31,386 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM"
2023-09-07 11:21:31,389 INFO sqlalchemy.engine.Engine [cached since 31.35s ago] ()
2023-09-07 11:21:31,395 INFO sqlalchemy.engine.Engine SELECT "ARTIST"."PK" AS "ARTIST_PK", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."PK" = ?
2023-09-07 11:21:31,396 INFO sqlalchemy.engine.Engine [cached since 44.5s ago] (1,)


[PK :2 - MyAlbum, 가수1]

In [176]:
artist1

2023-09-07 11:21:32,810 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE ? = "ALBUM"."FK"
2023-09-07 11:21:32,811 INFO sqlalchemy.engine.Engine [cached since 45.91s ago] (1,)


가수1, 1

In [174]:
Album.delAlbum(2)

2023-09-07 11:21:27,416 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."PK" = ?
2023-09-07 11:21:27,419 INFO sqlalchemy.engine.Engine [generated in 0.00291s] (1,)
2023-09-07 11:21:27,422 INFO sqlalchemy.engine.Engine DELETE FROM "ALBUM" WHERE "ALBUM"."PK" = ?
2023-09-07 11:21:27,424 INFO sqlalchemy.engine.Engine [generated in 0.00152s] (1,)
2023-09-07 11:21:27,427 INFO sqlalchemy.engine.Engine COMMIT


**TRY1 끝**

### 초기화해서 Relationship 다시 - 2

In [105]:
engine.dispose()
sess.close()
base.registry.dispose()
base.metadata.clear()

In [106]:
engine = create_engine('sqlite:///:memory:', echo=False)
base = declarative_base()
Session = sessionmaker(engine)
sess = Session()

In [107]:
#Artist 1 : N Album
#Post N:M HashTag =>관계를 정의하고 있는 별도의 테이블을 써야한다.
class Post(base):
    __tablename__ = 'POST'
    __table_args__ = { 'extend_existing': True }
    pk = Column('PK', Integer, primary_key=True)
    content = Column('CONTENT', Text)
    tags = relationship('Tags', back_populates='post', uselist=True)
    
    def addTags(self, *tags):
        tagList = list()
        for tag in tags:
            rst = sess.query(HashTag).filter(HashTag.name == tag)
            if rst.count() > 0:
                tagList.append(rst.one())
            else:
                newTag = HashTag(name = tag)
                sess.add(newTag)
                sess.commit()
                tagList.append(newTag)
        for tag in tagList:
            sess.add(Tags(fk1=self.pk, fk2=tag.pk))
            sess.commit()
            tag.cnt_increment()
            
    def delTags(self, *tags):
        tagList = list()
        for tag in tags:
            rst = sess.query(HashTag).filter(HashTag.name == tag)
            if rst.count() <= 0:
                continue
            else:
                res = rst.one()
                tagToDelete = sess.query(Tags).where(and_(Tags.fk1 == post3.pk, Tags.fk2 == res.pk)).one()
                sess.delete(tagToDelete)
                sess.commit()
                res.cnt_decrement()
    
    def __repr__(self):
        return f'PK:{self.pk}, CONTENT:{self.content},, TAGLEN:{len(self.tags)}'
    
class HashTag(base):
    __tablename__ = 'HASHTAG'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    cnt = Column('CNT', Integer, server_default='0')
    posts = relationship('Tags', back_populates='hashtag', uselist=True)
    
    def cnt_increment(self):
        self.cnt = self.cnt + 1
        
    def cnt_decrement(self):
        self.cnt = self.cnt - 1
    
    
    
    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}, POSTLEN:{len(self.posts)}, COUNT:{self.cnt}'
    
class Tags(base):
    __tablename__ = 'TAGS'
    pk = Column('PK', Integer, primary_key=True)
    fk1 = Column('FK1', Integer, ForeignKey('POST.PK'))
    fk2 = Column('FK2', Integer, ForeignKey('HASHTAG.PK'))
    post = relationship('Post', back_populates='tags')
    hashtag = relationship('HashTag', back_populates='posts')
    
    def __repr__(self):
        return f'PK{self.pk} : {self.fk1} -- {self.fk2}'
    
# Object/Class
# Post.tags[0].hashtag -------> HashTag

In [ ]:
def add_post(content, *tags):
    newpost = Post(content = content)
    

In [108]:
base.metadata.create_all(engine)

In [109]:
post1 = Post(content='내용1')
sess.add(post1)
sess.commit()

In [110]:
tag1 = HashTag(name = '해시태그1')
sess.add(tag1)
sess.commit()

In [111]:
sess.add(Tags(fk1=post1.pk, fk2=tag1.pk))
sess.commit()

In [112]:
len(post1.tags)

1

In [113]:
post1.tags[0].hashtag.name

'해시태그1'

In [114]:
len(tag1.posts)

1

In [115]:
tag1.posts[0].post.content

'내용1'

In [116]:
post2 = Post(content='내용2')
sess.add(post2)
sess.commit()

In [117]:
post2.addTags('해시태그1', '해시태그2', '해시태그3')

In [118]:
for tag in post2.tags:
    print(tag.hashtag)

PK:1, NAME:해시태그1, POSTLEN:2, COUNT:1
PK:2, NAME:해시태그2, POSTLEN:1, COUNT:1
PK:3, NAME:해시태그3, POSTLEN:1, COUNT:1


In [119]:
post3 = Post(content='내용3')
sess.add(post3)
sess.commit()

In [120]:
post3.addTags('해시태그1', '해시태그2', '해시태그4')

In [121]:
for tag in post3.tags:
    print(tag.hashtag)

PK:1, NAME:해시태그1, POSTLEN:3, COUNT:2
PK:2, NAME:해시태그2, POSTLEN:2, COUNT:2
PK:4, NAME:해시태그4, POSTLEN:1, COUNT:1


In [122]:
sess.query(Tags).all()

[PK1 : 1 -- 1,
 PK2 : 2 -- 1,
 PK3 : 2 -- 2,
 PK4 : 2 -- 3,
 PK5 : 3 -- 1,
 PK6 : 3 -- 2,
 PK7 : 3 -- 4]

In [123]:
sess.query(HashTag).all()

[PK:1, NAME:해시태그1, POSTLEN:3, COUNT:2,
 PK:2, NAME:해시태그2, POSTLEN:2, COUNT:2,
 PK:3, NAME:해시태그3, POSTLEN:1, COUNT:1,
 PK:4, NAME:해시태그4, POSTLEN:1, COUNT:1]

In [124]:
post3.delTags('해시태그1', '해시태그2')

In [125]:
post3

PK:3, CONTENT:내용3,, TAGLEN:1

In [126]:
sess.query(HashTag).all()

[PK:1, NAME:해시태그1, POSTLEN:2, COUNT:1,
 PK:2, NAME:해시태그2, POSTLEN:1, COUNT:1,
 PK:3, NAME:해시태그3, POSTLEN:1, COUNT:1,
 PK:4, NAME:해시태그4, POSTLEN:1, COUNT:1]